In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
EFFECT_SIZE_PATH = DATA_PATH.joinpath('capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023_gene_symbol_harmonized.pkl')
PROTEIN_SEQ_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')
EXON_PATH = DATA_PATH.joinpath('gene_symbol_dna_sequence_exon.pkl')
UNSPLICED_PATH = DATA_PATH.joinpath('gene_symbol_dna_sequence_unspliced.pkl')

In [2]:
df = pd.read_pickle(EFFECT_SIZE_PATH)

df = df.groupby(['gene_symbol_harmonized'], group_keys=False).apply(lambda x: x.loc[x['est_m_ea'] == x['est_m_ea'].max()])

assert not df['gene_symbol_harmonized'].duplicated().any()

df_effect_size = df.copy()

In [12]:
df = pd.read_pickle(PROTEIN_SEQ_PATH)

df = df[['gene_symbol_harmonized', 'seq']]

df = df.drop_duplicates()

df = df_effect_size.merge(df, how='inner', on='gene_symbol_harmonized')

df = df[['gene_symbol_harmonized', 'est_m_ea', 'seq']]

print(df.shape)

assert not df[['gene_symbol_harmonized', 'seq']].duplicated().any()

df.to_pickle(DATA_PATH.joinpath('protein_effect_size_analysis.pkl'))

df_protein_seq_effect_size = df.copy()

(23260, 3)


In [10]:
df = df_protein_seq_effect_size.copy()

train_genes = df['gene_symbol_harmonized'].drop_duplicates().sample(frac=.8)

df_train = df.loc[df['gene_symbol_harmonized'].isin(train_genes)]
df_test = df.loc[~df['gene_symbol_harmonized'].isin(train_genes)]

vectorizer = CountVectorizer(analyzer='char', ngram_range=(41,41))

X_train = vectorizer.fit_transform(df_train['seq'].tolist())
y_train = df_train['est_m_ea'].tolist()

X_test = vectorizer.transform(df_test['seq'].tolist())
y_test = df_test['est_m_ea'].tolist()

In [5]:
reg = LinearRegression().fit(X_train, y_train)

reg.score(X_test, y_test)

-0.011148760606491726

In [11]:
reg = KNeighborsRegressor().fit(X_train, y_train)

reg.score(X_test, y_test)

-0.0429357650205211

In [7]:
df = pd.read_pickle(PROTEIN_SEQ_PATH)

df = df.loc[df['motifs'].notna()]

df = df[['gene_symbol_harmonized', 'motifs']]

df = df.drop_duplicates()

df = df_effect_size.merge(df, how='inner', on='gene_symbol_harmonized')

df = df[['gene_symbol_harmonized', 'est_m_ea', 'motifs']]

print(df.shape)

assert not df[['gene_symbol_harmonized', 'motifs']].duplicated().any()

train_genes = df['gene_symbol_harmonized'].drop_duplicates().sample(frac=.8)

df_train = df.loc[df['gene_symbol_harmonized'].isin(train_genes)]
df_test = df.loc[~df['gene_symbol_harmonized'].isin(train_genes)]

vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,1))

X_train = vectorizer.fit_transform(df_train['motifs'].tolist())
y_train = df_train['est_m_ea'].tolist()

X_test = vectorizer.transform(df_test['motifs'].tolist())
y_test = df_test['est_m_ea'].tolist()

reg = LinearRegression().fit(X_train, y_train)

reg.score(X_test, y_test)

(588, 3)


-0.39686823117402836

In [8]:
# import sys
# import pathlib
# from subprocess import run

# project_path = pathlib.Path('/data1/home/adpatter/mouse-aging-deep-phenotyping/adpatter').resolve()

# if project_path.as_posix() not in sys.path: sys.path.append(project_path.as_posix())
# from utils.logger import logger as log

# project_path = pathlib.Path('/data1/home/adpatter/mouse-aging-deep-phenotyping/adpatter')
# run_path = project_path.joinpath('run')
# main_path = project_path.joinpath('main')

In [9]:
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('extract_unidimensional_experiment_20220324_173145.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('extract_categorical_experiment_20220324_173145.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('combine_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('harmonize_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment_wp1.ipynb')}", shell=True)
# run(f"jupyter nbconvert --to=python --output-dir={run_path} {main_path.joinpath('vector_transform_experiment_wp2.ipynb')}", shell=True)

In [10]:
# # log.info('Run extract_unidimensional_experiment_20220324_173145.py')
# # run(f"python {run_path}/extract_unidimensional_experiment_20220324_173145.py', shell=True)

# # log.info('Run extract_categorical_experiment_20220324_173145.py')
# # run(f"python {run_path}/extract_categorical_experiment_20220324_173145.py', shell=True)

# log.info('Run combine_experiment.py')
# run(f"python {run_path.joinpath('combine_experiment.py')}", shell=True)

# log.info('Run harmonize_experiment.py')
# run(f"python {run_path.joinpath('harmonize_experiment.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment_wp1.py')}", shell=True)

# log.info('Run vector_transform_experiment.py')
# run(f"python {run_path.joinpath('vector_transform_experiment_wp_2.py')}", shell=True)